In [261]:
import solana
from solana.publickey import PublicKey
from solana.rpc.api import Client
from solana.keypair import Keypair
from solana import system_program
from solana import transaction
from solana.system_program import SYS_PROGRAM_ID, CreateAccountParams, create_account
from solana.transaction import *
from spl.token.constants import TOKEN_PROGRAM_ID, ASSOCIATED_TOKEN_PROGRAM_ID
from spl.token.instructions import initialize_mint, InitializeMintParams
from spl.token._layouts import MINT_LAYOUT
from spl.token.client import Token
from spl.token import instructions
import base64
from borsh_construct import *
from solana.rpc.commitment import Commitment
import time


client = Client("https://api.devnet.solana.com")

client.is_connected()

True

In [262]:
program_id = PublicKey("9qm7AEJFHQ8SqJrmfofWK6maWRwKvQwK8uy8w3PVZLQw")
# program_id_account_meta = AccountMeta(program_id, False, False)
sys_program_meta = AccountMeta(SYS_PROGRAM_ID, False, False)
spl_program_meta  = AccountMeta(TOKEN_PROGRAM_ID, False, False)
# program_id_account_meta

In [263]:
# client.request_airdrop(PublicKey("DxkiJXK31VEd4R6F44mQRszoUQugpPYMwL9ybcrnt22r"), 2000000000)

In [264]:
def getAccountInfo(client: Client, account_pk: PublicKey):
    """Fetch account information from RPC, parse out the data and deserialize"""
    comm = Commitment("confirmed")
    res = client.get_account_info(account_pk, comm, encoding='base64')
    data = res['result']
    if isinstance(data, dict):
        return accountSchema.parse(base64.urlsafe_b64decode(data['value']['data'][0]))
    else:
        raise AttributeError(f'Unknown RPC result {data}')

In [265]:
seed = bytes([125,215,212,54,213,98,45,123]*4)
payer_keypair = Keypair.from_seed(seed)
client.request_airdrop(payer_keypair.public_key, 2000000000)

mint_account = Keypair.generate()

seed1 = bytes([225,215,211,54,213,18,25,122]*4)
vault_keypair = Keypair.from_seed(seed1)

payer_account_meta = AccountMeta(payer_keypair.public_key, True, True)
vault_account_meta = AccountMeta(PublicKey("G473EkeR5gowVn8CRwTSDop3zPwaNixwp62qi7nyVf4z"), False, True)

rent_account_meta = AccountMeta(solana.sysvar.SYSVAR_RENT_PUBKEY, False, False)
spl_program_meta  = AccountMeta(TOKEN_PROGRAM_ID, False, False)

# transaction = Transaction()
# transaction.add(
#   create_account(CreateAccountParams(from_pubkey=payer_account_meta.pubkey, new_account_pubkey=mint_account.public_key, lamports=Token.get_min_balance_rent_for_exempt_for_mint(client), space=MINT_LAYOUT.sizeof(), program_id=spl_program_meta.pubkey))
# )
# transaction.add(
#   initialize_mint(InitializeMintParams(
#     decimals=6, 
#     program_id=spl_program_meta.pubkey, 
#     mint=mint_account.public_key,
#     mint_authority=program_id, 
#     freeze_authority=program_id
#   ))
# )
   
# client.send_transaction(transaction, payer_keypair, mint_account)

In [266]:
mint_account_meta = AccountMeta(mint_account.public_key, True, True)
associated_token_address = instructions.get_associated_token_address(payer_keypair.public_key, mint_account.public_key)
associated_account_meta = AccountMeta(associated_token_address, False, True)

(mint_authority, nonce) = PublicKey.find_program_address([b'cyrial_pda'], program_id)
(locktime_pda, locktime_nonce) = PublicKey.find_program_address([b'locktime_pda', bytes(mint_account_meta.pubkey), bytes(associated_account_meta.pubkey)], program_id)
(sales_pda, sales_nonce) = PublicKey.find_program_address([b'sales_pda', bytes(program_id)], program_id)
metadata_program_pubkey = PublicKey("metaqbxxUerdq28cj1RbAWkYQm3ybzjb6a8bt518x1s")
(metadata_pda, metadata_nonce) = PublicKey.find_program_address([b'metadata', bytes(metadata_program_pubkey), bytes(mint_account_meta.pubkey)], metadata_program_pubkey)


locktime_pda_meta = AccountMeta(locktime_pda, False, True)
sysvar_clock_meta = AccountMeta(solana.sysvar.SYSVAR_CLOCK_PUBKEY, False, False)
mint_authority_meta = AccountMeta(mint_authority, False, False)

accounts = [
    # program_id_account_meta,
    payer_account_meta, 
    vault_account_meta, 
    mint_account_meta, 
    rent_account_meta, 
    associated_account_meta,
    spl_program_meta,
    AccountMeta(SYS_PROGRAM_ID, False, False),
    mint_authority_meta,
    locktime_pda_meta,
    AccountMeta(sales_pda, False, True),
    AccountMeta(metadata_pda, False, True),
    sysvar_clock_meta,

    spl_program_meta,
    AccountMeta(ASSOCIATED_TOKEN_PROGRAM_ID, False, False),
    spl_program_meta,
    spl_program_meta,
    AccountMeta(metadata_program_pubkey, False, False),
    ]

In [267]:
    
transaction = Transaction()
transaction.add(TransactionInstruction(
    accounts,
    program_id,
    bytes([0])
))
f"https://explorer.solana.com/tx/{client.send_transaction(transaction, payer_keypair, mint_account)['result']}?cluster=devnet"

'https://explorer.solana.com/tx/5dFmooura8tbSyYY4ET3WmMEVTToRFSmh8TR7DVL1tk7QwPw3t7BdzsSwnDqNHynybZrnaBCP1E6ynVs8e2QHVND?cluster=devnet'

In [269]:
new_accounts = [
    mint_account_meta,
    associated_account_meta,
    spl_program_meta,
    mint_authority_meta,
    locktime_pda_meta,
    sysvar_clock_meta,

    spl_program_meta
]

transaction = Transaction()
transaction.add(TransactionInstruction(
    new_accounts,
    program_id,
    bytes([1])
))
f"https://explorer.solana.com/tx/{client.send_transaction(transaction, payer_keypair, mint_account)['result']}?cluster=devnet"

'https://explorer.solana.com/tx/3n74xRxL8XKr21eHpeDjmDY8gt8BkqTBNQUU7gqyCQGF7qYuYfB4FtQ2FFHPMo5MiywXM4hE5N6BewrSVyFvR7iJ?cluster=devnet'

In [ ]:
time.sleep(5)
accountSchema = CStruct(
    "vault_total" / F32,
    "counter" / U32,
)

(sales_pda, sales_nonce) = PublicKey.find_program_address([b'sales_pda', bytes(program_id)], program_id)
print(f"PDA: {sales_pda, sales_nonce}")

getAccountInfo(client,sales_pda)

In [ ]:
accountSchema = CStruct(
    "date_created" / U32,
    "unlockable_data" / U32,
)


(locktime_pda, locktime_nonce) = PublicKey.find_program_address([b'locktime_pda', bytes(mint_account_meta.pubkey), bytes(associated_account_meta.pubkey)], program_id)
print(f"PDA: {locktime_pda, locktime_nonce}")

getAccountInfo(client,locktime_pda)

In [ ]:
# client.get_account_info(PublicKey("DEUptDVFvbPcYZCXS39ZAHMd6U8L2JmzPamSAJcovHJj"))